In [ ]:
import numpy as np

In [ ]:
class Neuron:
    def __init__(self, number_of_weights, bias, lr):
        self.weights = np.random.normal(0, 1, size=number_of_weights)
        self.bias = bias
        self.error = 0
        self.delta_weights = []
        self.delta_bias = 0
        self.lr = lr
    
    def calculate_neuron(self, inputs):
        """
        Calculate the output of this neuron
    
        """
        self.inputs = inputs
        list_outputs = []
        if len(inputs) == len(self.weights):
            for x, weight in zip(inputs, self.weights):
                list_outputs.append(x*weight)
        else:
            raise ValueError("The list of inputs and weights must have the same length, check if the neurons are well initialised")
        
        self.sum_output = sum(list_outputs)
        self.outcome = self.sum_output + self.bias
        output_sigmoid = self.sigmoid_function(self.outcome)
        return output_sigmoid
    
    def sigmoid_function(self, x):
        return 1/(1+np.exp(-x))
        
    def get_weights_and_error(self):
        return [self.weights, self.error]
    
    def update_neuron(self):
        for index in range(len(self.weights)):
            self.weights[index] -= self.delta_weights[index]
        self.bias-= self.delta_bias
        
        self.delta_weights = []
       
        #print('weights = ', self.weights, ' bias = ', self.bias, ' error = ', self.error)
            
class Output_neuron(Neuron):
    def calculate_error(self, output_neuron, target):
        self.error = output_neuron * (1 - output_neuron) * - (target - output_neuron)

    def gradient(self, input_x):
        return input_x * self.error
        
    def calculate_delta(self, output_neural_network):
        for weight, input_x in zip(self.weights, self.inputs):
            self.delta_weights.append(self.lr * self.gradient(input_x))
        self.delta_bias = self.lr * self.error
        
class Hidden_neuron(Neuron):
    def calculate_error(self, deltas_weights_right_layer):
        self.error += sum(deltas_weights_right_layer) 
        
    def calculate_delta(self): # let op dat je de error van de rechter layer pakt klopt dit?
        for weight, input_x in zip(self.weights, self.inputs):
            self.delta_weights.append(self.error * weight * input_x)
        self.delta_bias = self.bias - self.error 

In [ ]:
class Neuron_layer:
    def __init__(self, neurons):
        self.neurons = neurons
        self.hidden_errors = []
    
    def calculate_layer(self, inputs):
        output = []
        for neuron in self.neurons:
            output.append(neuron.calculate_neuron(inputs))
        return output
    
    def set_output_layer(self, outputNN, target):
        for neuron in self.neurons:
            neuron.calculate_error(outputNN, target)
            neuron.calculate_delta(outputNN)
    
    def set_hidden_layer(self, weights_and_error_right_layer): 
        for index in range(len(self.neurons)):
            self.neurons[index].calculate_error([info[0][index]*info[1] for info in weights_and_error_right_layer])
            self.neurons[index].calculate_delta()
    
    def get_weights_and_error(self):
        weights_and_error_layer = []
        for neuron in self.neurons:
            weights_and_error_layer.append(neuron.get_weights_and_error())
        return weights_and_error_layer
        
    def update_layer(self):
        for neuron in self.neurons:
            neuron.update_neuron()
    
    def __str__(self):
        return f'The layer consist out of these neurons: {self.neurons}'

In [ ]:
# test Neuron layer

def test_calculate_layer():
    test_neuron1 = Hidden_neuron(2, 0, 0.1)
    test_neuron2 = Hidden_neuron(2, 0, 0.1)
    test_neuron3 = Output_neuron(2, 0, 0.1)
    
    test_layer1 = Neuron_layer([test_neuron1, test_neuron2])
    test_layer2 = Neuron_layer([test_neuron3])
    
    test_layer1.calculate_layer()
    
    assert .404  == test_neuron.error, f' Error {test_neuron.error} does not match with the calculated values.'
    print('The calculate_error function works properly')


In [ ]:
class Neuron_network:
    def __init__(self, neuron_layers):
        self.neuron_layers = neuron_layers
    
    def feed_forward(self, inputs):
        #calculated the network by passing in inputs and reassign the output of the first layer to inputs it passes the output of layer A to layer B
        for neuron_layer in self.neuron_layers:
            inputs = neuron_layer.calculate_layer(inputs)
        return inputs
    
    def __str__(self):
        return f'The network consist out of these perceptron_layers: {self.neuron_layers}'
    
    def total_loss(self, target_list, output_list): # loss function 
        return np.sum([(a-b)**2 for a,b in zip(target_list, output_list)]) / 2 * len(target_list)
    
    def train_nn(self, posible_inputs):
        """
        Put in like this [[[0, 0], 0],
                          [[0, 1], 0],
                          [[1, 0], 0],
                          [[1, 1], 1]]
        """
        error = 1 # The error is unknown but will be assigned after the first check
        epoch = 0
        
        error_limit_value = 0.05
        while error > error_limit_value: #epoch < 10: #
            li = []
            for posible_input in posible_inputs:
                outputNN = self.feed_forward(posible_input[0])[0]
                li.append(outputNN)
            
                # backpropagation
                self.neuron_layers[-1].set_output_layer(outputNN, posible_input[1])
                
                for left_layer, right_layer in zip(self.neuron_layers[-2::-1], self.neuron_layers[-1::-1]): # Starting at -2 because last layer is output layer
                    weights_and_error_next_layer = right_layer.get_weights_and_error()
                    left_layer.set_hidden_layer(weights_and_error_next_layer)
                
                # update
                for layer in self.neuron_layers:
                    layer.update_layer()
            
            # check the difference between outcome of the network and the targets
            error = self.total_loss([x[1] for x in posible_inputs], li)
            print('error = ', error)
            if error < error_limit_value:
                print('network trained!')
                print('epochs = ', epoch)
            
            epoch+=1
            
            # shuffle the possible inputs in order to give better learn results
            #np.random.shuffle(posible_inputs)

In [ ]:
and_neuron = Output_neuron(2, 0, 0.2)

output_layer = Neuron_layer([and_neuron])

neural_network_AND = Neuron_network([output_layer])

In [ ]:
neural_network_AND.train_nn([[[0, 0], 0],
                             [[0, 1], 0],
                             [[1, 0], 0],
                             [[1, 1], 1]])

In [ ]:
inputs = [[[0,0],0],
          [[0,1],0],
          [[1,0],0],
          [[1,1],1]]

print('x1, x2,           output              expected_output')
for input_x in inputs:
    print(input_x, '   ', neural_network_AND.feed_forward(input_x[0])[0], '       ', input_x[1])

In [ ]:
nand_neuron_x = Hidden_neuron(2, 0, 0.2)
or_neuron_x = Hidden_neuron(2, 0, 0.2)
#extra_neuron_1 = Hidden_neuron(2, 0, 0.2, 3)
#extra_neuron_2 = Hidden_neuron(2, 0, 0.2, 4)

and_neuron_x = Output_neuron(2, 0, 0.2)


hidden_layer_x = Neuron_layer([nand_neuron_x, or_neuron_x])
output_layer_x = Neuron_layer([and_neuron_x])

neural_network_xor = Neuron_network([hidden_layer_x, output_layer_x])

In [ ]:
neural_network_xor.train_nn([[[0, 0], 0],
                             [[0, 1], 1],
                             [[1, 0], 1],
                             [[1, 1], 0]])

In [ ]:
print('done')

In [ ]:
inputs = [[[0,0],0],
          [[0,1],1],
          [[1,0],1],
          [[1,1],0]]

print('x1, x2,           output              expected_output')
for input_x in inputs:
    print(input_x, '   ', neural_network_xor.feed_forward(input_x[0])[0], '       ', input_x[1])

In [ ]:
# test functions for class Neuron

def test_calculate_neuron():
    test_neuron = Neuron(2, 0, 0.1)
    possible_inputs = [[[0, 0], 0],
                       [[0, 1], 0],
                       [[1, 0], 0],
                       [[1, 1], 1]]
    answer = [9.357622968839299e-14, 4.5397868702434395e-05, 4.5397868702434395e-05, 0.9999546021312976]
        
    for index, inputs in zip(range(len(possible_inputs)),possible_inputs):
        test_neuron.weights = [20, 20]
        test_neuron.bias = -30
        test_output = test_neuron.calculate_neuron(inputs[0])
        assert test_output == answer[index], f'Input {possible_inputs[index]} does not match with the calculated values.'
    print('The calculate_neuron function works properly')

def test_sigmoid_function():
    test_neuron = Neuron(2, 0, 0.1)
    test_output = test_neuron.sigmoid_function(5)
    assert test_output == 0.9933071490757153, 'There is something wrong with the formule of the sigmoidfunction.'
    print('The sigmoid_function works properly')

def test_get_weights_and_error():
    test_neuron = Neuron(2, 0, 0.1)
    test_neuron.weights = [0.5, 0.6]
    test_neuron.error = 0.22
    assert test_neuron.get_weights_and_error() == [[0.5, 0.6], 0.22], 'startingvalue of delta_weights are right'
    print('The get_weights_and_error function works properly')
    
def test_update_neuron():
    possible_inputs = [[[1, 1], 1]]
    test_neuron = Neuron(2, 0, 0.1)
    test_neuron.weights = [1,1]
    test_neuron.delta_weights = [0.1, 0.2]
    test_neuron.delta_bias = 0.3
    
    test_neuron.update_neuron()
    
    assert test_neuron.weights == [0.9, 0.8], f'After the update weights should be [0.9, 0.8] instead of {test_neuron.weights}'
    assert test_neuron.bias == -0.3, f'After the update the bias should be -0.3 instead of {test_neuron.bias}'
    print('The update_neuron function works properly')

test_sigmoid_function()    
test_calculate_neuron()
test_get_weights_and_error()
test_update_neuron()

In [ ]:
# test functions for class Output_neuron

def test_calculate_error():
    test_neuron = Output_neuron(2, 0, 0.1)
    test_neuron.calculate_error(0.3, 1)
    
    assert -0.147 == test_neuron.error, f' Error {test_neuron.error} does not match with the calculated values.'
    print('The calculate_error function works properly')

def test_gradient():
    test_neuron = Output_neuron(2, 0, 0.1)
    test_neuron.error = 0.25
    test_neuron.gradient(1)
    assert 0.25 == test_neuron.error, f' Error {test_neuron.error} does not match with the calculated values.'
    print('The calculate_error function works properly')

def test_calculate_delta():
    test_neuron = Hidden_neuron(2, 0, 0.1)
    test_neuron.weights = [0.5, 0.6]
    test_neuron.error = 0.22
    test_neuron.inputs = [1,1]
    test_neuron.calculate_delta()

    assert [0.11, 0.132] == test_neuron.delta_weights, f'Error deltaweights {test_neuron.delta_weights} does not match with [0.11, 0.32]'
    assert -0.22 == test_neuron.delta_bias, f'Error bias = {test_neuron.delta_bias} does not match with -0.22'
    print('The calculate_delta function works properly')
    
test_calculate_error()
test_gradient()
test_calculate_delta()

In [ ]:
# test functions for class Hidden_neuron

def test_calculate_error():
    test_neuron = Hidden_neuron(2, 0, 0.1)
    test_neuron.calculate_error([0.28, 0.124])
    
    assert .404  == test_neuron.error, f' Error {test_neuron.error} does not match with the calculated values.'
    print('The calculate_error function works properly')

def test_calculate_delta():
    test_neuron = Hidden_neuron(2, 0, 0.1)
    test_neuron.weights = [0.5, 0.6]
    test_neuron.error = 0.22
    test_neuron.inputs = [1,1]
    test_neuron.calculate_delta()

    assert [0.11, 0.132] == test_neuron.delta_weights, f'Error deltaweights {test_neuron.delta_weights} does not match with [0.11, 0.32]'
    assert -0.22 == test_neuron.delta_bias, f'Error bias = {test_neuron.delta_bias} does not match with -0.22'
    print('The calculate_delta function works properly')
    
test_calculate_error()    
test_calculate_delta()